In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
import re

# 設置 Selenium 驅動
options = Options()
options.add_argument("--headless")  # 如果需要顯示瀏覽器，請去掉此行
service = Service("/opt/homebrew/bin/chromedriver")  # 指定 ChromeDriver 的路徑
driver = webdriver.Chrome(service=service, options=options)

# 打開Google Travel的航班頁面
url = "https://www.google.com/travel/flights/booking?tfs=CBwQAhpKEgoyMDI0LTEwLTA0Ih4KA1RQRRIKMjAyNC0xMC0wNBoDTlJUKgJHSzICMTQoAGoMCAISCC9tLzBmdGt4cgwIAhIIL20vMDdkZmtAAUgBcAGCAQsI____________AZgBAg&tfu=CmhDalJJTFVwVE5tZHBTVGRGYW1kQlEwbGlkM2RDUnkwdExTMHRMUzB0TFhSaVltdHhPRUZCUVVGQlIySXlTV2RWU2tkeFEyZEJFZ1JIU3pFMEdnb0lpU01RQUJvRFZGZEVPQnh3eUc0PRICCAAiAA&authuser=0"
driver.get(url)

driver.implicitly_wait(10)

# 抓取出發日期
departure_date_element = driver.find_element(By.XPATH, "//span[contains(@class, 'mv1WYe')]").get_attribute("innerHTML")[:8]
departure_date = departure_date_element.strip()

# 抓取出發時間
departure_time_element = driver.find_element(By.XPATH, "//div[@class='wtdjmc YMlIz ogfYpf tPgKwe']").get_attribute("aria-label")
departure_time = departure_time_element.split("：")[-1].strip()  # 抓取時間部分

# 抓取抵達時間
arrival_time_element = driver.find_element(By.XPATH, "//div[@class='XWcVob YMlIz ogfYpf tPgKwe']").get_attribute("aria-label")
arrival_time = arrival_time_element.split("：")[-1].strip()  # 抓取時間部分

# 獲取所有符合條件的元素
airport_elements = driver.find_elements(By.XPATH, "//span[contains(@class, 'qeoz6e HKHSfd')]/following-sibling::span[@dir='ltr']")
# 抓取出發和抵達機場代碼
departure_airport = airport_elements[0].get_attribute("innerHTML").strip("()")  # 第一個是出發機場
arrival_airport = airport_elements[1].get_attribute("innerHTML").strip("()")    # 第二個是抵達機場

# 抓取航空公司
airline = driver.find_element(By.XPATH, "//div[contains(@class, 'sSHqwe')]/span[1]").text

# 抓取行程時間
travel_time_element = driver.find_element(By.XPATH, "//div[@class='CQYfx y52p7d WO360 QS0io']").get_attribute("innerHTML")
# 移除 "Travel time: " 前缀（如果存在）
flight_duration = travel_time_element.replace("路程時間：", "").strip()

# 抓取機型
aircraft = driver.find_element(By.XPATH, '//div[@class="MX5RWe sSHqwe y52p7d"]/span[@class = "Xsgmwe"][last()]').get_attribute("innerHTML")

# 抓取航班代碼
flight_number_element = driver.find_element(By.XPATH, '//div[@class="MX5RWe sSHqwe y52p7d"]/span[contains(@class, "Xsgmwe")][2]').get_attribute("innerHTML")
flight_number = flight_number_element.replace('&nbsp;', ' ').strip()  # 去除前後空白

# 抓取艙等
cabin_class = driver.find_element(By.XPATH, '//span[contains(@class, "Xsgmwe")]/div').get_attribute("innerHTML")

# 輸出結果
print(f"出發日期: {departure_date}")
print(f"出發時間: {departure_time}")
print(f"出發機場代號: {departure_airport}")
print(f"抵達時間: {arrival_time}")
print(f"抵達機場代號: {arrival_airport}")
print(f"航空公司: {airline}")
print(f"飛行時間: {flight_duration}")
print(f"機型: {aircraft}")
print(f"航班代碼: {flight_number}")
print(f"艙等: {cabin_class}")


# 抓取價格
price_element = driver.find_element(By.XPATH, "//div[contains(@class, 'frOi8 BgYkof fVSoi')]")
price = price_element.text
print(price)

# 抓取價格範圍
least_expensive_flight_element = driver.find_element(By.XPATH, "//div[contains(@class, 'NtS4zd eoY5cb')]")
range = least_expensive_flight_element.text
print(range)

# 抓取價格歷史
elements = driver.find_elements(By.XPATH, "//*[name()='g' and @class='ke9kZe-LkdAo-RbRzK-JNdkSc pKrx3d']")
price_history = [element.get_attribute("aria-label") for element in elements]
for entry in price_history:
    print(entry)

# 匯出至 CSV
with open('google_flights_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # 寫入每個屬性及其值
    writer.writerow(["出發日期", departure_date])
    writer.writerow(["出發時間", departure_time])
    writer.writerow(["出發機場代號", departure_airport])
    writer.writerow(["抵達時間", arrival_time])
    writer.writerow(["抵達機場代號", arrival_airport])
    writer.writerow(["航空公司", airline])
    writer.writerow(["飛行時間", flight_duration])
    writer.writerow(["機型", aircraft])
    writer.writerow(["航班代碼", flight_number])
    writer.writerow(["艙等", cabin_class])
    writer.writerow(["價格", price])
    writer.writerow(["價格範圍", range])
    
    # 寫入價格歷史
    writer.writerow(["價格歷史", ""])
    for entry in price_history:
        writer.writerow(["", entry])  # 每個歷史價格在新行中
        
# 關閉瀏覽器
driver.quit()


出發日期: 10月4日 週五
出發時間: 下午1:10。
出發機場代號: TPE
抵達時間: 下午5:35。
抵達機場代號: NRT
航空公司: 捷星日本航空
飛行時間: 3 小時 25 分鐘
機型: Airbus A320
航班代碼: GK 14
艙等: 經濟艙
以經濟艙而言，$4,789屬於一般價位
飛往東京都的相似行程最低航班票價通常介於 $3,850 - 5,900 之間。
60 天前 - $5,089
59 天前 - $4,789
58 天前 - $4,489
57 天前 - $4,489
56 天前 - $4,489
55 天前 - $4,489
54 天前 - $4,489
53 天前 - $4,489
52 天前 - $4,489
51 天前 - $4,489
50 天前 - $4,489
49 天前 - $4,489
48 天前 - $4,489
47 天前 - $4,489
46 天前 - $4,489
45 天前 - $4,489
44 天前 - $4,489
43 天前 - $4,489
42 天前 - $4,189
41 天前 - $4,489
40 天前 - $4,489
39 天前 - $4,489
38 天前 - $4,489
37 天前 - $4,489
36 天前 - $4,489
35 天前 - $4,189
34 天前 - $4,189
33 天前 - $4,189
32 天前 - $4,189
31 天前 - $4,189
30 天前 - $4,189
29 天前 - $4,189
28 天前 - $4,189
27 天前 - $4,189
26 天前 - $4,189
25 天前 - $4,189
24 天前 - $4,189
23 天前 - $4,189
22 天前 - $4,189
21 天前 - $4,189
20 天前 - $4,189
19 天前 - $3,589
18 天前 - $3,589
17 天前 - $3,589
16 天前 - $3,589
15 天前 - $3,589
14 天前 - $3,589
13 天前 - $3,889
12 天前 - $3,889
11 天前 - $3,889
10 天前 - $4,189
9 天前 - $4,189
8 天前 - $4,189
7 天前 - $4,189
